In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import re
from utils import *
from layers import *
from AdvASLTM import *
import keras_tuner
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer

2023-03-15 18:35:01.485898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 18:35:05.454425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/clement/miniconda3/envs/tf/lib/
2023-03-15 18:35:05.454948: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/clement/miniconda3/envs/tf/lib/
2023-03-15 18:35:05.454963: W tensorflow/compiler/tf

In [3]:
nrows = 150000
x_train_df = pd.read_csv("input_training.csv", index_col="ID")
y_train_df = pd.read_csv("output_training_gmEd6Zt.csv", index_col="ID")
x_test_df = pd.read_csv("input_test.csv", index_col="ID")

if nrows is not None:
    x_train_df = x_train_df.iloc[-nrows:]
    y_train_df = y_train_df.iloc[-nrows:]

y_train_df["reod"] = y_train_df["reod"] + 1

train_df = x_train_df.join(y_train_df).copy(deep=True)

In [4]:
r_filter = re.compile("r[0-9]+")
features_columns = list(filter(r_filter.match, train_df.columns)) # + ["week_day", "total_r0-52"]
preprocessed_features_columns = [f"preprocessed_{col}" for col in features_columns]

# Cleaning Data
1 - Dealing with NaNs

In [ ]:
# Check distribution of label on row that contain NaN
train_df.loc[(train_df.isna().sum(axis=1) > 10), "reod"].value_counts()

In [5]:
#As NaNs are also present in the test dataset, I cannot delete. I chose to replace them with zeros
train_df.replace(np.nan, 0, inplace = True)
x_test_df.replace(np.nan, 0, inplace = True)

2 - Plot data

In [ ]:
train_df["total_r0-52"] = list(map(
    composed_bps_returns,
    train_df[list(filter(r_filter.match, train_df.columns))].to_numpy()
    ))

In [ ]:
#plot_matrix(train_df, "day", "equity", "total_r0-52", lambda x : list(x)[0])

# Add features
1 - Time features (day of the week)

In [ ]:
train_df["week_day"] = train_df["day"].apply(lambda x : x % 5)
#plot_matrix(train_df,"week_day", "equity", "total_r0-52", lambda x : list(x)[0], x_lim=20)

# Preprocess

In [6]:
train_df.sort_values(by="day", inplace= True) # Sort by day to get as close as possible from the test dataset conditions with my validation dataset

train_ratio = 0.8
train_limit = int(train_ratio*len(train_df))

X_train = train_df[features_columns].iloc[:train_limit].to_numpy()
y_train = tf.keras.utils.to_categorical(train_df['reod'].iloc[:train_limit], num_classes=3)

X_validation = train_df[features_columns].iloc[train_limit:].to_numpy()
y_validation = tf.keras.utils.to_categorical(train_df['reod'].iloc[train_limit:], num_classes=3)

X_test = x_test_df[features_columns].to_numpy()

# Model

In [11]:

def reccurent_layer(type, *args, **kwargs):
    if type == "LSTM":
        return tf.keras.layers.LSTM(*args, **kwargs)
    elif type == "GRU":
        return tf.keras.layers.GRU(*args, **kwargs)
    raise ValueError(f"Type args is wrong : '{type}' while it should be 'LSTM' or 'GRU'")

def optimizer(type, *args, **kwargs):
    if type == "adam":
        return tf.keras.optimizers.Adam(*args, **kwargs)
    elif type == "rmsprop":
        return tf.keras.optimizers.RMSprop(*args, **kwargs)
    raise ValueError(f"Type args is wrong : '{type}' while it should be 'adam' or 'rmsprop'")


class MyHyperModel(keras_tuner.HyperModel):
    def __init__(self, X_train, y_train, X_validation, y_validation, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.X_train = X_train
        self.X_validation = X_validation
        self.y_train = y_train
        self.y_validation = y_validation

        self.tranformer_robust = RobustScaler(unit_variance = True, with_centering= True).fit(X_train)
        self.tranformer_quantile = QuantileTransformer(output_distribution='normal').fit(X_train)
        self.tranformer_power = PowerTransformer(method='yeo-johnson', standardize=True).fit(X_train)

        self.X_train_robust = self.tranformer_robust.transform(X_train)[:, :, np.newaxis]
        self.X_train_quantile = self.tranformer_quantile.transform(X_train)[:, :, np.newaxis]
        self.X_train_power = self.tranformer_power.transform(X_train)[:, :, np.newaxis]
        self.X_validation_robust = self.tranformer_robust.transform(X_validation)[:, :, np.newaxis]
        self.X_validation_quantile = self.tranformer_quantile.transform(X_validation)[:, :, np.newaxis]
        self.X_validation_power = self.tranformer_power.transform(X_validation)[:, :, np.newaxis]



    
    def build(self, hp):
        nb_reccurrent_layers = hp.Int("nb_reccurrent_layers", min_value = 1, max_value = 4, step=1)
        reccurrent_unit = hp.Int("reccurrent_unit", min_value = 4, max_value = 512, sampling = "log")
        reccurrent_dropout = hp.Float("reccurrent_dropout", min_value = 0, max_value = 0.3, sampling = "linear")
        reccurent_type = hp.Choice("reccurent_type", ["LSTM", "GRU"])
        attention_layer = hp.Boolean("attention_layer")
        learning_rate = hp.Float("learning_rate",  1E-5, 1E-2, sampling = "log")
        opt_type = hp.Choice("optimizer", ["adam", "rmsprop"])
        
        layers = []
        
        for i in range(nb_reccurrent_layers - 1):
            layers.append(reccurent_layer(reccurent_type, reccurrent_unit, dropout = reccurrent_dropout, return_sequences = True))
        
        if attention_layer:layers.append(AttentionLayer(reccurrent_unit, dropout = reccurrent_dropout))
        else:layers.append(reccurent_layer(reccurent_type, reccurrent_unit, dropout = reccurrent_dropout, return_sequences = False))

        
        layers.append(tf.keras.layers.Dense(3, activation= "softmax"))

        model = tf.keras.models.Sequential(layers)
        
        model.compile(
            optimizer = optimizer(opt_type, learning_rate = learning_rate),
            loss = hp.Choice("loss", ["categorical_crossentropy", "categorical_hinge"]), 
            metrics=["acc"]
        )
        
        return model
    
    
    def fit(self, hp, model, *args, **kwargs):
        preprocessing_method = hp.Choice("preprocess", ["robust_scale", "power_transform", "quantile_transform", "none"])
        if preprocessing_method == "robust_scale":
            X_train = self.X_train_robust
            X_validation = self.X_validation_robust
        elif preprocessing_method == "quantile_transform":
            X_train = self.X_train_quantile
            X_validation = self.X_validation_quantile
        elif preprocessing_method == "power_transform":
            X_train = self.X_train_power
            X_validation = self.X_validation_power
        else:
            raise ValueError(f"Preprocessing method is invalid : '{preprocessing_method}'")
            
        seq_len = hp.Int("seq_len", min_value = 5, max_value = X_train.shape[1], step=1)

        
        return model.fit(
            X_train[:, -seq_len:, :], self.y_train,
            validation_data = (X_validation[:, -seq_len:, :], self.y_validation),
            *args, **kwargs
        )

In [ ]:
tuner = keras_tuner.Hyperband(
    MyHyperModel(X_train, y_train, X_validation, y_validation),
    objective="val_acc",
    max_epochs=30,
    factor=2,
    hyperband_iterations=1,
    directory="keras_tuner_checkpoints",
    project_name="tune_hypermodel2",
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, mode='min')

In [ ]:
tuner.search(callbacks = [stop_early])

In [ ]:
#import shutil
#shutil.rmtree('/notebooks/keras_tuner_checkpoints/tune_hypermodel')

In [ ]:
dict_best_hp = { key:[] for key in tuner.get_best_hyperparameters()[0].values.keys()}
list_best_hp = tuner.get_best_hyperparameters(num_trials= 10)
for i in range(10):
    for key in dict_best_hp.keys():
        dict_best_hp[key].append(list_best_hp[i].values[key] if key in list_best_hp[i].values.keys() else None)
pd.DataFrame(dict_best_hp)

# Test best models

In [7]:
tranformer_robust = RobustScaler(unit_variance = True, with_centering= True).fit(X_train)

X_train_robust = tranformer_robust.transform(X_train)[:, :, np.newaxis]
X_validation_robust = tranformer_robust.transform(X_validation)[:, :, np.newaxis]


seq_len = 40
dropout = 0.3

In [6]:
inputs = tf.keras.layers.Input(X_train_robust[:, -seq_len:, :].shape[1:])
x = tf.keras.layers.Dense(32, activation = "tanh", kernel_regularizer = tf.keras.regularizers.L2(0.01))(inputs )
x = tf.keras.layers.Dropout(0.3)(x)
h = tf.keras.layers.LSTM(64, return_sequences= True, dropout= 0.4)(x)
alpha = TemporalAttentionLayer(units = 64)(h)
x = LatentLayer()([alpha,h])
x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation = "relu", kernel_regularizer = tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Dropout(0.3)(x)
output = tf.keras.layers.Dense(3, activation = "softmax")(x)



model = tf.keras.models.Model(inputs = inputs, outputs = output)
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1E-4),
    loss= "categorical_crossentropy",
    metrics =["acc"]
)


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model_weights/Model_{epoch:02d}-{val_loss:.4f}-{val_acc:0.4f}.hdf5",
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=12,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)

model.fit(
    X_train_robust[:, -seq_len:, :], y_train,
    validation_data = (X_validation_robust[:, -seq_len:, :], y_validation),
    batch_size= 64,
    epochs = 200,
    callbacks=[model_checkpoint_callback, early_stop]
)

2023-03-15 17:19:11.424599: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 17:19:11.792130: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 17:19:11.792306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-15 17:19:11.879081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Epoch 1/200


2023-03-15 17:19:39.701941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-15 17:19:42.073153: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f18d801a4f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-15 17:19:42.088774: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2023-03-15 17:19:42.317805: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-15 17:19:43.746911: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-15 17:19:43.992296: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1875/1875 [==============================] - 63s 22ms/step - loss: 1.7584 - acc: 0.4208 - val_loss: 1.1944 - val_acc: 0.4572
Epoch 2/200
1875/1875 [==============================] - 38s 20ms/step - loss: 1.1113 - acc: 0.4559 - val_loss: 1.0532 - val_acc: 0.4787
Epoch 3/200
1875/1875 [==============================] - 41s 22ms/step - loss: 1.0590 - acc: 0.4553 - val_loss: 1.0379 - val_acc: 0.4796
Epoch 4/200
1875/1875 [==============================] - 43s 23ms/step - loss: 1.0497 - acc: 0.4550 - val_loss: 1.0396 - val_acc: 0.4804
Epoch 5/200
1875/1875 [==============================] - 37s 20ms/step - loss: 1.0453 - acc: 0.4566 - val_loss: 1.0294 - val_acc: 0.4827
Epoch 6/200
1875/1875 [==============================] - 42s 22ms/step - loss: 1.0423 - acc: 0.4575 - val_loss: 1.0270 - val_acc: 0.4824
Epoch 7/200
1875/1875 [==============================] - 43s 23ms/step - loss: 1.0409 - acc: 0.4579 - val_loss: 1.0283 - val_acc: 0.4835
Epoch 8/200
1875/1875 [==============================

KeyboardInterrupt: 

In [8]:
X_test_robust = tranformer_robust.transform(X_test)[:, :, np.newaxis]
X_test_robust.shape

(885799, 53, 1)

In [10]:
new_model = tf.keras.models.load_model('model_weights/Model_16-1.0200-0.4844.hdf5', 
    custom_objects={
        'TemporalAttentionLayer': TemporalAttentionLayer,
        'LatentLayer':LatentLayer
    })
X_test_robust = tranformer_robust.transform(X_test)[:, :, np.newaxis]
results = new_model.predict(X_test_robust[:, -seq_len:, :], batch_size = 64)

: 

: 

In [ ]:
x_test_df["reod"] = results.argmax(axis=-1) -1
result_df = x_test_df[["reod"]]
result_df

In [ ]:
result_df.to_csv("y_test.csv")

In [ ]:
result_df.to_csv("y_test.csv")